In [2]:
pip install ucimlrepo


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
  
# fetch online retail dataset 
online_retail = fetch_ucirepo(id=352) 


In [6]:
# original df
df = online_retail.data.original

# drop country column
df = df.drop(columns='Country')

# get rid of any invoices with unit price <= 0 and stockcode doesn't start with a number or DCG as these are not sales but stock movements, expenses, etc
df = df[df['UnitPrice'] > 0]
df = df[df['StockCode'].str.contains('^[0-9]|DCG', regex = True)]

# find most common description for each stock code
df['Description'] = df['Description'].str.strip(' ')
correct_descriptions = df.groupby('StockCode')['Description'].agg(pd.Series.mode)
mapping = dict(correct_descriptions)

#add to map items which do not have a most common used description
mapping['35817P'] = 'PINK ACRYLIC JEWEL SNOWFLAKE'
mapping['90014C'] = 'SILVER AND BLACK ORBIT NECKLACE'

# use the map to update the description column
df['Description'] = df['StockCode'].map(mapping)

# strip description from unnecessary characters
df['Description'] = df['Description'].str.strip('*')

# add a new column for sale value
df['SaleValue'] = df.UnitPrice * df.Quantity

# check for nulls
print('Total null values per column:')
print(df.isnull().sum())

# df[df['CustomerID'].isna()].to_csv('report.csv')
df.to_csv('report.csv')

Total null values per column:
InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     131586
SaleValue           0
dtype: int64
